##### Please download dataset at https://www.kaggle.com/mahmoudreda55/satellite-image-classification?select=data and put in folder named "image"

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

2022-02-01 20:14:28.580345: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
img_size=128
def preprocess(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(img_size,img_size),interpolation=cv2.INTER_NEAREST)
    img = np.array(img)
    img = img.astype("float32")
    img= img/255.0
    return img

In [3]:
path = "image"
x = []
y = []
for label in os.listdir(path):
    subdir = os.path.join(path,label)
    for fname in os.listdir(subdir):
        img = cv2.imread(os.path.join(subdir,fname))
        img = preprocess(img)
        x.append(img)
        y.append(label)

In [4]:
x = np.array(x)
y = np.array(y)

In [5]:
ec = OneHotEncoder()
y = ec.fit_transform(y.reshape(-1,1)).toarray()

In [6]:
x.shape, y.shape

((5631, 128, 128, 3), (5631, 4))

In [8]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(input_shape=(img_size, img_size, 3), filters=32, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),padding="same", activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=100, activation="relu"))
model.add(tf.keras.layers.Dense(units=y.shape[-1], activation="softmax"))

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)


In [10]:
#Train model
history = model.fit(
    x, y,
    batch_size=64, epochs=50,
    verbose=1
)

2022-02-01 20:16:40.364193: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 1107099648 exceeds 10% of free system memory.


Epoch 1/50


2022-02-01 20:16:41.073473: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 134217728 exceeds 10% of free system memory.
2022-02-01 20:16:41.119727: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 134217728 exceeds 10% of free system memory.
2022-02-01 20:16:41.170791: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 33554432 exceeds 10% of free system memory.
2022-02-01 20:16:41.181781: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 67108864 exceeds 10% of free system memory.


28/88 [========>.....................] - ETA: 1:13 - loss: 7.9803 - accuracy: 0.3331

KeyboardInterrupt: 